In [262]:
import pandas as pd
import datetime
import plotly
import plotly.express as px
import plotly.graph_objects as go

#chartTemplate = 'plotly_dark'
chartTemplate = 'plotly_white'

In [281]:
cases = pd.read_csv("C:/Users/Admin/Documents/GitHub/Tutoriales/Covid19/covid19-casos.csv")

#cases = pd.read_json("C:/Users/Admin/Documents/GitHub/Tutoriales/Covid19/covid19-casos.json")

cases_cfg = cases[cases["provincia"] == "Cienfuegos"]

cuba = cases['fecha_confirmacion'].value_counts().reset_index().rename(columns={'index': 'Date','fecha_confirmacion':'N'}).sort_values(by='Date', ascending=True)
cfg = cases_cfg['fecha_confirmacion'].value_counts().reset_index().rename(columns={'index': 'Date','fecha_confirmacion':'N'}).sort_values(by='Date', ascending=True)

# calculate case velocity
cuba["vel"] = cuba.loc[:, cuba.columns != 'Date'].diff()
cfg["vel"] = cfg.loc[:, cfg.columns != 'Date'].diff()

# calculate case acceleration
cuba["acc"] = cuba[["Date", "vel"]].loc[:, cuba[["Date", "vel"]].columns != 'Date'].diff()
cfg["acc"] = cfg[["Date", "vel"]].loc[:, cfg[["Date", "vel"]].columns != 'Date'].diff()

# calculate means - 14 días - 2 semanas
# usando rolling
avg = 14
cfg["accelMeans14"] = cfg[["Date", "vel"]].loc[:, cfg[["Date", "vel"]].columns != 'Datetime'].rolling(window = avg).mean()
cuba["accelMeans14"] = cuba[["Date", "vel"]].loc[:, cuba[["Date", "vel"]].columns != 'Datetime'].rolling(window = avg).mean()
# con EWM
# cfg["accelMeans5"] = cfg[["Date", "vel"]].loc[:, cfg[["Date", "vel"]].columns != 'Datetime'].ewm(span=5).mean()

cfg

C:\Python39\lib\site-packages\IPython\core\interactiveshell.py:3169: DtypeWarning:

Columns (1,7) have mixed types.Specify dtype option on import or set low_memory=False.



,Date,N,vel,acc,accelMeans14
231,2020/03/24,1,NaN,NaN,NaN
195,2020/03/27,3,2.0,NaN,NaN
217,2020/03/28,1,-2.0,-4.0,NaN
200,2020/03/29,3,2.0,4.0,NaN
197,2020/04/01,3,0.0,-2.0,NaN
...,...,...,...,...,...
5,2021/06/30,79,13.0,34.0,2.785714
3,2021/07/01,86,7.0,-6.0,1.285714
2,2021/07/02,87,1.0,-6.0,4.285714
7,2021/07/03,73,-14.0,-15.0,2.428571


In [282]:
fig = px.line(cfg, x="Date", y="N", template=chartTemplate)
fig.update_layout({"title": 'Casos - Cienfuegos',
                   "xaxis": {"title":"Días"},
                   "yaxis": {"title":"Casos"},
                   "showlegend": False})

fig.show()

In [283]:
fig = px.line(cfg, x="Date", y="vel", template=chartTemplate)

fig.update_layout({"title": 'Velocidad de casos - Cienfuegos',
                   "xaxis": {"title":"Días"},
                   "yaxis": {"title":"Casos"}})
                   #"showlegend": False})
ffig.show()

In [284]:
cfg.head(5)

,Date,N,vel,acc,accelMeans14
231,2020/03/24,1,NaN,NaN,NaN
195,2020/03/27,3,2.0,NaN,NaN
217,2020/03/28,1,-2.0,-4.0,NaN
200,2020/03/29,3,2.0,4.0,NaN
197,2020/04/01,3,0.0,-2.0,NaN


In [285]:
import numpy as np
# here you define the color
#cfg["color"] = np.where(cfg["accelMeans14"] >= 0, 'green', "red")
## add a color column to the df, apply along row
cfg['color'] = cfg.apply(lambda x: 'red' if x['accelMeans14'] >= 0 else 'green', axis=1)
pos = cfg[cfg['accelMeans14'] >= 0]
neg = cfg[cfg['accelMeans14'] < 0]

fig = go.Figure()

fig = px.line(cfg, x="Date", y="accelMeans14", template=chartTemplate)

#fig.add_trace(go.Bar(x=cfg['Date'], y=cfg['accelMeans14'], name='Promedio de 14 días', marker = dict(color = 'red')) 

fig.add_trace(go.Bar(x=pos['Date'], y=pos['accelMeans14'], name='Promedio de 14 días', marker = dict(color = 'red')))
fig.add_trace(go.Bar(x=neg['Date'], y=neg['accelMeans14'], name='Promedio de 14 días', marker = dict(color = 'green')))#
fig.update_layout({"title": 'Aceleración de casos - promedio 14 días - Cienfuegos',
                   "xaxis": {"title":"Días"},
                   "yaxis": {"title":"Casos"},

                   "width" : 1000,
                   "height" : 800,                   "showlegend": False})
fig.show()

#  ##--

# CUBA

## Análisis de Cuba

#  ##--

In [286]:
fig = px.line(cuba, x="Date", y="N", template=chartTemplate)

fig.update_layout({"title": 'Casos - Cuba',
                   "xaxis": {"title":"Días"},
                   "yaxis": {"title":"Casos"},
                   "showlegend": False})

fig.show()

In [287]:
fig = px.line(cuba, x="Date", y="vel", template=chartTemplate)

fig.update_layout({"title": 'Velocidad - Cuba',
                   "xaxis": {"title":"Días"},
                   "yaxis": {"title":"Casos"},
                   "showlegend": False})

fig.show()

In [288]:
fig = px.line(cuba, x="Date", y="acc", template=chartTemplate)

fig.update_layout({"title": 'Aceleración de casos - Cuba',
                   "xaxis": {"title":"Días"},
                   "yaxis": {"title":"Casos"},
                   "showlegend": False})

fig.show()

In [289]:
fig = px.line(cuba, x="Date", y="accelMeans14", template=chartTemplate)

fig.update_layout({"title": 'Aceleración de casos - promedio 14 días - Cuba',
                   "xaxis": {"title":"Días"},
                   "yaxis": {"title":"Casos"},
                   "showlegend": False})
fig.add_trace(go.Bar(x=cuba['Date'], y=cuba['accelMeans14'], name='Promedio de 14 días'))

fig.show()